# Overview

## Lecture 8: fMRI hemodynamic response function and convolution

In this session, we will learn about the hemodynamic response function and how *convolve* a signal with this function.

In the last few weeks, we computed event-related averages for responses to different categories of objects. We saw that responses in fMRI emerge slowly (2-3 TRs or 4-6 seconds) after the onset of a stimulus.

In an fMRI experiment, we usually would like to know how some event is related to brain responses and where in the brain we can find representations of these events. Using the event related averages we can investigate which brain regions respond more to a particular stimuli than others. This week, we will introduce how we can model the slow nature of the fMRI signal. To drive any valid conclusions from the fMRI data we need to make sure that our events model the actual changes in the brain. By modeling the delay that is inate to the fMRI signal, we can later try to find a direct mapping between the event and the brain response (which we will see in in the lectures after the Spring break).

# Goals for today

We will go over some important concepts of how to incorporate the delan in the fMRI signal after a stimulus (or event) onset 

- Neuroscience concepts
    - Modeling hemodynamic responses
- Coding concepts
    - Implementing convolution
- Datascience concepts
    - Time series convolution
    - Correlation 

In [ ]:
# Update neurods
# !!! Once updated remember to restart the jupyter notebook kernel !!!
import neurods as nds
nds.io.update_neurods()

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

# Configure defaults for plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.cmap'] = 'viridis'

# New defaults!
from cycler import cycler
plt.rcParams['axes.grid'] = True
plt.rcParams['axes.prop_cycle'] = cycler('color', ['k','r', (0,0,0.8),(0.95,0.8,0),(0,.8,0),(1.0, 0.5, 0)])

In [ ]:
%matplotlib inline
# %config InlineBackend.figure_format = 'retina' # prettier plots for retina screens (optional)

# fMRI responses do not occur immediately after a stimulus

fMRI responses emerge slowly after the onset of a stimulus (or any other event). This means that, if we want to make an accurate mathematical model of how the brain responds to a given stimulus, we have to incorporate this delay into our model. 

To do this, we will borrow a concept from signal processing theory called the impulse response function. An impulse response function describes a system's response (this can be the signal that we measure using fMRI) to an external change (e.g. the stimulus or event onset). As we know, fMRI measures BOLD signal and not neuronal activity directly. Therefore, to model the hemodynamic response (BOLD signal) to an event we would like to have a function that describes how the BOLD signal looks like when it responds to a given event. This function is called the *hemodynamic response function* or HRF. 

A great deal of early fMRI research went into accurately describing how the BOLD signal rises, falls, and resets to baseline after an event. We will rely on the conclusions of this research without going into much detail about it. For an overview of hemodynamic responses in fMRI, check out [this blog post](http://mindhive.mit.edu/node/72), and the papers in the `figures/` directory for this lecture (Handwerker, Bandettini et al 2012; Logothetis & Wandell, 2004). The practical upshot of this work is that BOLD responses have a fairly characteristic shape, which is well described by a mathematical function called the *gamma function*. 


#### Generate and plot the hemodynamic response function

Below we will use the fmri module in the neurods package to generate a canonical HRF:

`neurods.fmri.hrf()`

In [ ]:
# Let's check what arguments this function takes
import neurods
neurods.fmri.hrf??

In [ ]:
# Import a function that generates an HRF
from neurods.fmri import hrf as generate_hrf

# Set the TR, or repetition time, a.k.a. the sampling rate
TR = 1.0 # One measurement per second

# Get a canonical HRF
t_hrf, hrf_1 = generate_hrf(tr=TR)
print('hrf_1 shape is', hrf_1.shape)
__ = plt.plot(t_hrf, hrf_1)
plt.title("Canonical HRF")

#### Plot a single discrete stimulus

In [ ]:
# Plot a single discrete stimulus that appears at time 0
t = t_hrf
stimulus = np.zeros(t.shape)
stimulus[0] = 1

plt.figure(1, figsize=(10, 5))
plt.subplot(121)
plt.plot(t, stimulus)

# There is a better function that we can use to make this plot more explicit
plt.subplot(122)
plt.stem(t, stimulus, linefmt='k-', markerfmt='.', basefmt='k-', label='Stimulus')

#### The HRF is the BOLD response to this single discrete stimulus

Let's plot the HRF together with the stimulus

In [ ]:
# We will be plotting stimulus / response pairs several times
# Hence, here is a function that can plot these two together
def stim_resp_plot(t, stimulus, response, yl=(-0.2, 1.2), label_stim='Stimulus', label_resp='BOLD response (HRF)'):
    """Plot stimulus and response."""
    plt.figure(figsize=(10,4))
    plt.stem(t, stimulus, linefmt='k-', markerfmt='.', basefmt='k-', label=label_stim)
    plt.plot(t, response, 'r.-', label=label_resp)
    plt.ylim(yl)
    plt.xlim([-1,t.max()+1])
    plt.xlabel('Time (seconds)')
    plt.ylabel('Response (arbitrary units)')
    _ = plt.legend()

# Plot
stim_resp_plot(t, stimulus, hrf_1)

### Breakout session
Play with the parameters of the `neurods.fmri.hrf()` function. See what happens if you change them. 

In [ ]:
### STUDENT ANSWER
# The notation (*neurods.fmri.hrf())
# in the plot function unzips the tuple (timepoints, hrf) that was returned by neurods.fmri.hrf()
# timepoints, hrf = neurods.fmri.hrf()
plt.plot(*neurods.fmri.hrf())
plt.plot(*neurods.fmri.hrf(pttp=5, tr=1))
plt.plot(*neurods.fmri.hrf(pttp=8))
plt.plot(*neurods.fmri.hrf(pttp=8, tr=2))

Every time an event occurs, the response we measure is similar to this slow hemodynamic response. We will slowly explore how this hemodynamic response can affect the signal in a voxel.

First, we start with a hypothetical run of 200 TRs, in which no stimuli is presented.

In [ ]:
n = 200 # Total time points (TRs)
t = np.arange(n,)

# No stimulus
stimuli = np.zeros((n))

plt.figure(figsize=(10,4))
plt.xlabel('Time (seconds)')
plt.ylabel('Response (arbitrary units)')
plt.stem(t, stimuli)

# We assume no response
response = np.zeros((n))

# Here we plot the function
stim_resp_plot(t, stimuli, response, yl=(-0.2, 1.2))

### Breakout session (a)

A few cells above we plotted how the signal changes when we have a stimulus at time 0. 

Now, imagine we have a stimulus at time `i=10`. What do you expect will happen when we plot the stimulus and response?

Hint: This stimulus will create an HRF that will be *added* to the signal from times i to times i + hrf_length (hrf_length is the length of our canonical HRF, which was 32 above).

Attention: Make sure you modify the response by adding something to it's values, and not only changing it.

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = len(hrf_1)
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
response = np.zeros((n))

# Add a stimlus onset at time 10
i = 10
stimuli[i] = 1

plt.figure(figsize=(10,4))
plt.xlabel('Time (seconds)')
plt.ylabel('Response (arbitrary units)')
plt.stem(t, stimuli)

# Now add the response to stimulus i to the response, then plot using stim_resp_plot like above
### STUDENT ANSWER
response[range(i,i+hrf_length)] += hrf_1
stim_resp_plot(t, stimuli, response, yl=(-0.2, 1.2))

### Breakout session (b)


Now let's say that there were 3 event onsets, one at `i=10`, one at `i=70` and one at `i=150`, plot the resulting activity. 

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = len(hrf_1)
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
response = np.zeros((n))

stim_times = [10, 70, 150]
for i in stim_times:
    stimuli[i] = 1
    
plt.figure(figsize=(10,4))
plt.xlabel('Time (seconds)')
plt.ylabel('Response (arbitrary units)')
plt.stem(t, stimuli)

# Now add the response to these three events, then plot using stim_resp_plot like above
### STUDENT ANSWER
for i in stim_times:
    response[range(i,i+hrf_length)] += hrf_1
stim_resp_plot(t, stimuli, response, yl=(-0.2, 1.2))

### Breakout session (c)

Now say that the stimuli are closer together than the length of the hemodynamic function: let's say they occur at times 10, 21, 25, 70, 71,74, 75, 80 and 150, what happens? 

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = len(hrf_1)
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
response = np.zeros((n))

stim_times = [10, 21, 25, 70, 75, 80, 150]
for i in stim_times:
    stimuli[i] = 1

plt.figure(figsize=(10,4))
plt.xlabel('Time (seconds)')
plt.ylabel('Response (arbitrary units)')
plt.stem(t, stimuli)

# Now add the response to these events to the response, then plot using stim_resp_plot like above
### STUDENT ANSWER
for i in stim_times:
    response[range(i,i+hrf_length)] += hrf_1
stim_resp_plot(t, stimuli, response, yl=(-0.2, 1.2))

### Breakout session (d)
What happens if the event appears at time `i=180`? 

Hint: You may need to make a change involving the min function.

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = 32
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
response = np.zeros((n))

stim_times = [180]
for i in stim_times:
    stimuli[i] = 1

plt.figure(figsize=(10,4))
plt.xlabel('Time (seconds)')
plt.ylabel('Response (arbitrary units)')
plt.stem(t, stimuli)

# Now add the response to this event to the response, then plot using stim_resp_plot like above
### STUDENT ANSWER
for i in stim_times:
    index = range(i, min(i+hrf_length, n))
    response[index] += hrf_1[:len(index)]
stim_resp_plot(t, stimuli, response, yl=(-0.2, 1.2))

#plt.figure()
#plt.plot(hrf_1)
#plt.plot(hrf_1[:len(index)])

### Breakout session (e)

Now write a function that will produce the correct response vectors given the stimulus vector and the hrf_1 vector. Make sure that the function will go over every element in the stimulus vector. You might need to do an adjustment to the last breakout session (d).

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = len(hrf_1)
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))

stim_times = [10, 21, 25, 70, 75, 80, 150]
for i in stim_times:
    stimuli[i] = 1

plt.figure(figsize=(10,4))

plt.figure(figsize=(10,4))
plt.xlabel('Time (seconds)')
plt.ylabel('Response (arbitrary units)')
plt.stem(t, stimuli)
    
# Define your function here, then use it to generate output and plot the above plots:
def gen_responses(stimulus_vec, hrf_canonical):
    n = stimulus_vec.shape[0]
    hrf_length = len(hrf_canonical)
    response = np.zeros((n,))
    for i in range(n):
### STUDENT ANSWER
        if stimulus_vec[i]==1: # one way to do it: only if I have a stimulus, I will add a response
            index = range(i, min(i+hrf_length, n))
            response[index] += hrf_canonical[:len(index)]
    return response

response = gen_responses(stimuli,hrf_1)
stim_resp_plot(t, stimuli, response, yl=(-0.2, 1.2))